note to jesse the residue number changes when make new pdb!!

note to jessi: append list of file paths

In [13]:
import time
from collections import Counter
from collections import defaultdict
from itertools import chain, product
o = time.perf_counter()

#give cutout/titratable residue neighborhood, an id
#residues = ['GLU', 'ASP', 'TYR', 'CYS', 'HIS'] #no arg or lys cuz none other state and arg none in db!
lines_2_edit, static_lines, input_pdb, i, n_old, grouped_res_to_edit, all_states =[],[], "11as_B_204_LYS", 0, 0, [], []

split_fname = input_pdb.split("_")[1:3] 
cutoutid=(int(split_fname[1]), split_fname[0]) #chain, resname 

with open("11as") as file: #open the protein residue file which gives all the residues which are in my protein #HERE JESSI USEFUL FOR CHANGING CUTOUT GENERATION SCHEME
    all_residues = (file.read()).split("'), (")
pdb_residues = [(int(residue[0]), residue[1]) for residue in [line.split(", '") for line in all_residues]]

with open(".".join([input_pdb,"pdb"])) as file: #READ THE PDB
    lines=file.readlines()[:-2] #list of all the lines
for line in lines: 
    split_lines = line.split()
    number, Chain, name = split_lines[5], split_lines[4], split_lines[3][:2] 
    ID_res = (int(number), Chain)
    if ID_res in pdb_residues and ID_res != cutoutid and name[1] != "R" and name[0] != "L": #no arginine and LYS #if the line is included in the residues which have a cutout, and isnt the ML datapoint
        lines_2_edit.append(line)
    else:
        static_lines.append(line)

for n in dict(dict(Counter(list(([int(line.split()[5]) for line in lines_2_edit]))).items()).items()).values(): #LIST(..) is list of residue numbers[299,299,299,235,235]
    i = i + n_old
    grouped_res_to_edit.append(lines_2_edit[i:i+n])
    n_old = n 

resmap_combos, resmap_singles, asp_states, his_states, glu_states, i = {"A": ["ASH", "AS4"], "H": ["HIE", "HID", "HIP"], "G": ["GL4", "GLH"]}, {"C": "CYM", "T":"TYM"}, [], [], [], 0

for Res in grouped_res_to_edit:
    res = Res[0].split()[3] #the first line
    first_letter = res[0]
    plines = [line.replace(res, resmap_singles[res]) for line in Res if first_letter == 'C'] #cys
    if plines:
        plines.append([line.replace(res, resmap_singles[res]) for line in Res if first_letter == 'T']) #TYR
    else:
        plines = [line.replace(res, resmap_singles[res]) for line in Res if first_letter == 'T'] #TYR

    if first_letter=="A": #asp
        new = resmap_combos[first_letter]
        asp_states.append([line.replace(res, new[0]) for line in Res])
        asp_states.append([line.replace(res, new[1]) for line in Res])
        all_states.append(asp_states)
    if first_letter=="G": #glu
        new = resmap_combos[first_letter]
        glu_states.append([line.replace(res, new[0]) for line in Res])
        glu_states.append([line.replace(res, new[1]) for line in Res])
        all_states.append(glu_states)
    if first_letter=="H": #his
        new = resmap_combos[first_letter]
        his_states.append([line.replace(res, new[0]) for line in Res])
        his_states.append([line.replace(res, new[1]) for line in Res])
        his_states.append([line.replace(res, new[2]) for line in Res])
        all_states.append(his_states)

keep, i = static_lines + plines, 0 #keep the protonatable lines of cys and TYR, and the non-titratable residue lines
 
all_combos = list(product(*all_states)) #makes all the unique combinations of acidic residues 
for combos in all_combos[:2]:
    combined_list = list(chain(*combos, keep)) #grab from the combos bag once and combine it with all the static lines to make final pdb-to-be-protonated

    with open(f"{input_pdb}_{i}.pdb", "w") as file: #write the protonated acidic pdb
        file.writelines(combined_list)
    
#pdbfilename = "11as_248_B.pdb"
#make and save the script to run in amber which will protonate the pdb (1/total combos) 
skript = f"""source /Users/jessihoernschemeyer/miniconda3/envs/cfcnn/dat/leap/cmd/leaprc.constph
mol = loadpdb "/Users/jessihoernschemeyer/pKaSchNet/{input_pdb}_{i}.pdb"
savepdb mol "/Users/jessihoernschemeyer/pKaSchNet/{i}.pdb"
quit"""
with open("ascript.py","w") as file: 
    file.writelines(skript)

#execute the script
f = time.perf_counter()
for i in range((2)):
    !tleap -s -f /Users/jessihoernschemeyer/pKaSchNet/ascript.py
#print(f-o, "seconds", len(all_combos))


-I: Adding /Users/jessihoernschemeyer/miniconda3/envs/cfcnn/dat/leap/prep to search path.
-I: Adding /Users/jessihoernschemeyer/miniconda3/envs/cfcnn/dat/leap/lib to search path.
-I: Adding /Users/jessihoernschemeyer/miniconda3/envs/cfcnn/dat/leap/parm to search path.
-I: Adding /Users/jessihoernschemeyer/miniconda3/envs/cfcnn/dat/leap/cmd to search path.
-s: Ignoring all leaprc startup files.
-f: Source /Users/jessihoernschemeyer/pKaSchNet/ascript.py.

Welcome to LEaP!
Sourcing: /Users/jessihoernschemeyer/pKaSchNet/ascript.py
----- Source: /Users/jessihoernschemeyer/miniconda3/envs/cfcnn/dat/leap/cmd/leaprc.constph
----- Source of /Users/jessihoernschemeyer/miniconda3/envs/cfcnn/dat/leap/cmd/leaprc.constph done
----- Source: /Users/jessihoernschemeyer/miniconda3/envs/cfcnn/dat/leap/cmd/oldff/leaprc.ff10
----- Source of /Users/jessihoernschemeyer/miniconda3/envs/cfcnn/dat/leap/cmd/oldff/leaprc.ff10 done
Log file: ./leap.log
Loading parameters: /Users/jessihoernschemeyer/miniconda3/envs

In [9]:
!cat 11as_B_204_LYS_0.pdb

ATOM     25  CA  HID B 209      47.390  64.803  11.319  1.00  2.00           C  
ATOM     26  N   HID B 209      47.573  66.217  11.585  1.00 17.30           N  
ATOM     27  C   HID B 209      46.899  64.100  12.567  1.00  5.99           C  
ATOM     28  O   HID B 209      47.481  64.247  13.654  1.00  2.00           O  
ATOM      1  CD  LYS B 204      41.049  66.412  11.518  1.00 42.23           C  
ATOM      2  CG  LYS B 204      42.577  66.352  11.726  1.00 20.94           C  
ATOM      3  NZ  LYS B 204      38.859  65.190  11.863  1.00 16.98           N  
ATOM      4  CE  LYS B 204      40.310  65.264  12.227  1.00 26.04           C  
ATOM      5  CB  LYS B 204      43.041  66.981  13.062  1.00 19.63           C  
ATOM      6  CA  LYS B 204      43.342  65.967  14.174  1.00 14.81           C  
ATOM      7  N   LYS B 204      42.099  65.418  14.705  1.00 17.86           N  
ATOM      8  C   LYS B 204      44.178  66.604  15.282  1.00 12.62           C  
ATOM      9  O   LYS B 204  

tuples are more efficient because they are immutable

In [66]:
%%time
lines_2_edit, static_lines =[],[]
with open(input_pdb) as file:
    lines=file.readlines()[:-2] #list of all the lines


for line in lines: #still gonna operate one line at a time if they meet the condition, dont be the cutout name, must be in input_pdb
    #resname = line.split()[3]
    #if resname in residues:
    split_lines = line.split()
    number, chain, name = split_lines[5], split_lines[4], split_lines[3][:2] 
    ID_res = (int(number), chain)
    if ID_res in pdb_residues and ID_res != cutoutid and name[1] != "R" and name[0] != "L": #arginine #if the line is included in the residues which have a cutout, and isnt the ML datapoint
        lines_2_edit.append(line)
    else:
        static_lines.append(line)


CPU times: user 2.59 ms, sys: 2.01 ms, total: 4.6 ms
Wall time: 3.9 ms


In [67]:
%%time
from collections import Counter
from collections import defaultdict

# Count occurrences
#counts = dict(Counter(list(([int(line.split()[5]) for line in lines_2_edit]))).items())



# Convert defaultdict to a regular dictionary to maintain the order
#countss = dict(dict(Counter(list(([int(line.split()[5]) for line in lines_2_edit]))).items()).items())



i, n_old, grouped_res_to_edit = 0, 0, []
for n in dict(dict(Counter(list(([int(line.split()[5]) for line in lines_2_edit]))).items()).items()).values(): #LIST(..) is list of residue numbers[299,299,299,235,235]

    i = i + n_old

    #res_lines = lines_2_edit[i:i+n]
    grouped_res_to_edit.append(lines_2_edit[i:i+n])
    n_old = n 




CPU times: user 356 µs, sys: 4 µs, total: 360 µs
Wall time: 373 µs


In [60]:

resmap_combos = {"A": ["ASH", "AS4"], "H": ["HIE", "HID", "HIP"], "G": ["GL4", "GLH"]}
resmap_singles = {"C": "CYM", "T":"TYM"}


In [76]:
%%time


plines=[]
all_states=[]

for Res in grouped_res_to_edit:
    res = Res[0].split()[3] #the first line
    first_letter = res[0]
    #print(Res)
    plines = [line.replace(res, resmap_singles[res]) for line in Res if first_letter == 'C'] #cys
    if plines:
        plines.append([line.replace(res, resmap_singles[res]) for line in Res if first_letter == 'T']) #TYR
    else:
        plines = [line.replace(res, resmap_singles[res]) for line in Res if first_letter == 'T'] #TYR





#resmap_combos = {"ASP": ["ASH", "AS4"], "HIS": ["HIE", "HID", "HIP"], "GLU": ["GL4", "GLH"]}
    asp_states, his_states, glu_states=[],[],[]
    if first_letter=="A":
        asp_states.append([line.replace(res, resmap_combos[first_letter][0]) for line in Res])
        asp_states.append([line.replace(res, resmap_combos[first_letter][1]) for line in Res])
        all_states.append(asp_states)
    if first_letter=="G":
        glu_states.append([line.replace(res, resmap_combos[first_letter][0]) for line in Res])
        glu_states.append([line.replace(res, resmap_combos[first_letter][1]) for line in Res])
        all_states.append(glu_states)
    if first_letter=="H":
        his_states.append([line.replace(res, resmap_combos[first_letter][0]) for line in Res])
        his_states.append([line.replace(res, resmap_combos[first_letter][1]) for line in Res])
        his_states.append([line.replace(res, resmap_combos[first_letter][2]) for line in Res])
        all_states.append(his_states)


keep = static_lines + plines


CPU times: user 144 µs, sys: 3 µs, total: 147 µs
Wall time: 158 µs


In [75]:
%%time
from itertools import chain, product
all_combos = list(product(*all_states))
combos=all_combos[0]

combined_list = list(chain(*combos, keep))

with open("apdb.pdb", "w") as file:
    file.writelines(combined_list)

CPU times: user 219 µs, sys: 531 µs, total: 750 µs
Wall time: 758 µs


worrrrkkkkkssss hypeeeeeeeeeeeeee

In [63]:


pdbfilename = "11as_248_B.pdb"
skript = f"""source /Users/jessihoernschemeyer/miniconda3/envs/cfcnn/dat/leap/cmd/leaprc.constph
mol = loadpdb "/Users/jessihoernschemeyer/pKaSchNet/{pdbfilename}"
savepdb mol "/Users/jessihoernschemeyer/pKaSchNet/out.pdb"
#quit"""

with open("ascript.py","w") as file:
    file.writelines(skript)

In [64]:

def if_asp(his1,his2,his3):
    h1a1 = his1 + asp_states[0]
                    
    h1a2 = his1  + asp_states[1]

    h2a1 = his2  + asp_states[0]

    h2a2 = his2  + asp_states[1]

    h3a1 = his3  + asp_states[0]

    h3a2 = his3  + asp_states[1]
    return [h1a1, h1a2, h2a1, h2a2, h3a1, h3a2]

def if_glu(h1a1, h1a2, h2a1, h2a2, h3a1, h3a2):
    g0 = h1a1 + glu_states[0] 
    g1 = h1a1 + glu_states[1] 
                    
    g2 = h1a2  + glu_states[0] 
    g3 = h1a2  + glu_states[1]
    g4 = h2a1  + glu_states[0]
    g5 = h2a1 + glu_states[1]

    g6 = h2a2   + glu_states[0]
    g7 = h2a2  + glu_states[1]

    g8 = h3a1   + glu_states[0]
    g9 = h3a1   + glu_states[1]

    g10 = h3a2  + glu_states[0]
    g11 = h3a2  + glu_states[1]
    return [g0,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,g11]
#if_glu(ifasp(his1,his2,his3))



plines=[]
all_states=[]

#this really gotta go one residue at a time

for Res in grouped_res_to_edit:
    res = Res[0].split()[3] #the first line
    first_letter = res[0]
    #print(Res)
    plines = [line.replace(res, resmap_singles[res]) for line in Res if first_letter == 'C'] #cys
    if plines:
        plines.append([line.replace(res, resmap_singles[res]) for line in Res if first_letter == 'T']) #TYR
    else:
        plines = [line.replace(res, resmap_singles[res]) for line in Res if first_letter == 'T'] #TYR

    #at this point, plines should be part of an acidic pdb.
#protonabable lines ^



#resmap_combos = {"ASP": ["ASH", "AS4"], "HIS": ["HIE", "HID", "HIP"], "GLU": ["GL4", "GLH"]}
    asp_states, his_states, glu_states=[],[],[]
    if first_letter=="A":
        print('a')
        asp_states.append([line.replace(res, resmap_combos[first_letter][0]) for line in Res])
        asp_states.append([line.replace(res, resmap_combos[first_letter][1]) for line in Res])
        all_states.append(asp_states)
    if first_letter=="G":
        print('b')
        glu_states.append([line.replace(res, resmap_combos[first_letter][0]) for line in Res])
        glu_states.append([line.replace(res, resmap_combos[first_letter][1]) for line in Res])
        all_states.append(glu_states)
    if first_letter=="H":
        his_states.append([line.replace(res, resmap_combos[first_letter][0]) for line in Res])
        his_states.append([line.replace(res, resmap_combos[first_letter][1]) for line in Res])
        his_states.append([line.replace(res, resmap_combos[first_letter][2]) for line in Res])
        all_states.append(his_states)

#now the ones with combos..
        
    #plines = [line.replace(res, "hi") for line in Res if first_letter == 'A'] #dummy single state lines
    
#DELETTTE AS WE GOO
#deal with his, then asp, then glu i.g.?
#i guess im working... per pdb, acquire all the protonatedlines, 
"""if not plines:
    plines=[]"""


if his_states:
    his1, his2, his3 = his_states[0], his_states[1], his_states[2]


    if asp_states:

        #h1a1, h1a2, h2a1, h2a2, h3a1, h3a2 = if_asp(his1,his2,his3)

        if glu_states:
            final_states = if_glu(if_asp(his1,his2,his3))
            print(final_states)
        else:
            final_states = if_asp(his1,his2,his3)
            print(final_states)


    else: #no asp states
        if glu_states:
            final_states= [his1 + glu_states[0], his1 + glu_states[1],his2 + glu_states[0], his2 + glu_states[1],his3 + glu_states[0], his3 + glu_states[1]]
            print(final_states)
        
        else: #no glu states, no asp states
        #g1,g2,g6 = if_glu(his1, his1, his2, his2, his3, his3)
            final_states = [his1, his2, his3]
            print(final_states)

else: # no his
    if asp_states:
        if glu_states:
            final_states = [asp_states[0] + glu_states[0],
                            asp_states[0] + glu_states[1],
                            asp_states[1] + glu_states[0],
                            asp_states[1]+glu_states[1]]
            print(final_states)
        else: #no his and glu
            final_states = [asp_states[0], asp_states[1]]
            print(final_states)

    else: #no asp (+no his)
        if glu_states:
            final_states = [glu_states[0], glu_states[1]]
            print((final_states))
        else: #no his glu or asp #DONT NEED THIS CUZ THEN WE HAVE NO MULTIPROTONATABLE RES.
            #all_protonated_final.append(final_states)
            print(final_states)
            pass
    #print(final_states)


#if glu_states:
    #del glu_states[:2]
#else:
    #if asp_states:
        #del asp_states[:2]
    #else:
        #if his_states:
            #del his_states[:3]
#[list + keep for list in final_states] #instead of list can interatively save
#success!!!!!!!!
#[list for list in final_states]

#save 



    #his_asp_combos = ["","","","","",""]
    #g0,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,g11 = if_glu(his_asp_combos)

#else:
    #h1a1, h1a2, h2a1, h2a2, h3a1, h3a2 = [],[],[],[],[],[]

#if glu_states:
    #g0,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,g11 = if_glu(h1a1, h1a2, h2a1, h2a2, h3a1, h3a2)
#else:
    #g1,g2,g6 = if_glu(h1a1, h1a2, h2a1, h2a2, h3a1, h3a2)
            
            
    #remove asp_states
    #his2 = plines + static_lines  + his_states[1]
    #his3 = plines + static_lines  + his_states[2]

    #THIS IS NO GONNA WORK BC I CAN HAVE MULTIPLE ASP STATES HENCE INDEXING DOESNT WORK.

#Yes it can? bc itll be like [Ash 299 ash 299]




a
a
a
b
b
a
a
[['ATOM    191  CD2 HIE B 211      47.311  57.554  18.120  1.00 10.44           C  \n', 'ATOM    192  NE2 HIE B 211      48.488  56.901  18.427  1.00 12.38           N  \n', 'ATOM    193  CE1 HIE B 211      49.507  57.657  18.075  1.00  9.44           C  \n'], ['ATOM    191  CD2 HID B 211      47.311  57.554  18.120  1.00 10.44           C  \n', 'ATOM    192  NE2 HID B 211      48.488  56.901  18.427  1.00 12.38           N  \n', 'ATOM    193  CE1 HID B 211      49.507  57.657  18.075  1.00  9.44           C  \n'], ['ATOM    191  CD2 HIP B 211      47.311  57.554  18.120  1.00 10.44           C  \n', 'ATOM    192  NE2 HIP B 211      48.488  56.901  18.427  1.00 12.38           N  \n', 'ATOM    193  CE1 HIP B 211      49.507  57.657  18.075  1.00  9.44           C  \n']]
